# Take Home Assignment for Data Engineer Intern
## Context

Steven is a travel blogger that intends to create a travel food series. He is looking at data from Zomato for inspiration. He wants to find restaurants that have good user ratings and interesting past events. This assignment involves quering the json file provided and accessing 3 items. 

(1) Restaurant Information

(2) Restaurant Events Information

(3) Aggregate Rating Information

This notebook will detail the methodology of querying information from the json file provided.

## Importing Dependencies
The main libraries in use will be 

(1) `requests` to obtain the json information

(2) `pandas` represent the queries in a DataFrame Format

In [13]:
import requests
import pandas as pd

## Loading the Dataset

In [14]:
data = requests.get("https://raw.githubusercontent.com/Papagoat/brain-assessment/main/restaurant_data.json").json()

## Step 1: Extract Restaurant Data

Extraction of Restaurant Data from the json file provided

The features of the restaurant information have also been extracted as the respective columns as shown in the code

In [15]:
#Create DataFrame with the respective columns as stated in the requirements file
df = pd.DataFrame(columns = ['rest_id',
            'name',
            'country',
            'city',
            'rating_votes',
            'agregate_rating',
            'cuisines'])

#for all the results available in the data
for results in data:
    # for all the restaurants within each result
    for rest in results['restaurants']:
        dic = {
            'rest_id' : rest["restaurant"]["id"],
            'name' : rest["restaurant"]["name"],
            'country': rest["restaurant"]["location"]["country_id"],
            'city': rest["restaurant"]["location"]["city_id"],
            'rating_votes': rest["restaurant"]["user_rating"]["votes"],
            'agregate_rating': rest["restaurant"]["user_rating"]["aggregate_rating"],
            'cuisines': rest["restaurant"]['cuisines']
        }
        #upload the respective information into the DataFrame
        df.loc[len(df)] = dic

### Addition of Country Information into the DataFrame

In [16]:
#Change country_id to country name
country = pd.read_excel("Country-Code.xlsx")

In [28]:
df["country"] = df["country"].apply(lambda x: country.loc[country["Country Code"].values == x, "Country"].values[0] if x in country["Country Code"].values else None)

df.to_csv("restaurant.csv", sep = ",", index = False)
df.head()

,rest_id,name,country,city,rating_votes,agregate_rating,cuisines
0,18649486,The Drunken Botanist,None,1,4765,4.4,"Continental, Italian, North Indian, Chinese"
1,308322,Hauz Khas Social,None,1,13627,4.6,"Continental, American, Asian, North Indian, Ch..."
2,18856789,AIR- An Ivory Region,None,1,1819,4.1,"North Indian, Chinese, Continental, Asian"
3,307374,AMA Cafe,None,1,3252,4.4,"Cafe, Juices"
4,18238278,Tamasha,None,1,8112,4.4,"Finger Food, North Indian, Continental, Italian"


## Step 2
Extract the list of restuarants events that occured in the month of April 2019

The features of the events have also been extracted as the respective columns as shown in the code

In [27]:
from datetime import datetime

#initialising DataFrame with the respective columns as provided by the task
df1 = pd.DataFrame(columns=['rest_id', 'name', 'photo_url', 'event_id', 'event_title', 'start_date', 'end_date'])

#for all the results in data
for results in data:
    #for each restaurant in the result
    for rest in results['restaurants']:
        #common features throughout same restaurants and different restaurant events
        dic = {
            'rest_id' : rest["restaurant"]["id"],
            'name' : rest["restaurant"]["name"],
            'photo_url' : rest["restaurant"]["photos_url"]
        }
        entry = dic.copy()
        #check if zomato_events is in the restaurant data / does the restaurant have events
        if "zomato_events" in rest["restaurant"]:
            for event in rest["restaurant"]["zomato_events"]:
                #Record the start date, year and month & end date, year and month
                start_date = datetime.strptime(event["event"]["start_date"], "%Y-%m-%d")
                start_month = start_date.month
                start_year = start_date.year
                end_date = datetime.strptime(event["event"]["end_date"], "%Y-%m-%d")
                end_month = end_date.month
                end_year = end_date.year
                #check that it is during April 2019
                if start_month <= 4 <= end_month and start_year <= 2019 <= end_year:
                    entry["event_id"] = event["event"]["event_id"] if "event_id" in event["event"] else "NA"
                    entry["event_title"] = event["event"]["title"] if "title" in event["event"] else "NA"
                    entry["start_date"] = start_date 
                    entry["end_date"] = end_date
                    
                    #Add each entry into the dataframe
                    df1.loc[len(df1)] = entry

df1.to_csv("restaurant_events.csv", sep = ",", index = False)

In [25]:
df1.head()
df1.shape

(163, 7)

## Step 3
Determine the threshold for the different rating text based on aggregate rating

In [20]:
dic = {'Excellent' : [],
       'Very Good' : [],
       'Good' : [],
       'Average' : [],
       'Poor' : [],
}

#Include a whole list of aggregate rating score according to the rating text
for results in data:
    for rest in results["restaurants"]:
        text = rest["restaurant"]["user_rating"]["rating_text"]
        if text in dic:
            dic[text].append(float(rest["restaurant"]["user_rating"]["aggregate_rating"]))

for key in dic:
    temp = [0] * 2
    #Calculate the max and min of each list
    max_rating = max(dic[key])
    min_rating = min(dic[key])
    temp[0] = min_rating
    temp[1] = max_rating
    dic[key] = temp
print(dic)

{'Excellent': [4.5, 4.9], 'Very Good': [4.0, 4.4], 'Good': [3.5, 3.9], 'Average': [2.5, 3.4], 'Poor': [2.2, 2.2]}
